In [1]:
"""
Check and activate Nvidia GPU

"""
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))


display("Number of GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
display(device_lib.list_local_devices())

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 970, pci bus id: 0000:01:00.0, compute capability: 5.2



'Number of GPUs Available: '

1

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16109232823816068962,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2959205990
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13035064519604526319
 physical_device_desc: "device: 0, name: GeForce GTX 970, pci bus id: 0000:01:00.0, compute capability: 5.2"]

In [2]:
"""
Import all the reqired modules

"""
# %load_ext autoreload
# from importlib import reload

import pandas as pd
import numpy as np
import tensorflow as tf

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Bidirectional
from tensorflow.keras.layers import Flatten, GRU, RNN, SimpleRNN
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import losses
from numpy import asarray

In [4]:
# change the dir of your CSV as well as the X, y columns for prediction

data_path = 'c:/users/ezalgo/all-assignments/assignment6/data/data-p2v2.csv'
df = pd.read_csv(data_path)
y = df['label']
X = df['nltk_stem']


In [5]:
# split the set using sklearn's train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=192)

In [6]:
max_features=50000

docs = X_train
#display(len(docs))
t = Tokenizer(num_words=50000)
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(docs)
max_length = 350
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
word_index = t.word_index
print('Found %s unique tokens.' % len(word_index))

Found 160224 unique tokens.


In [7]:
# configure the stanford glove directory and the model used

# load the whole embedding into memory
embeddings_index = dict()
f = open('glove/glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

num_words = min(max_features, len(word_index)) + 1
print(num_words)

embedding_dim = 100

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)
embedding_matrix.shape

Loaded 400000 word vectors.
50001


(50001, 100)

In [8]:
"""
embedding glove - lstm
https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/
"""


DROPOUT_RATE = 0.2

model = Sequential()
model.add(Embedding(max_features+1, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))

model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(BatchNormalization())
model.add(Dropout(DROPOUT_RATE))

model.add(Bidirectional(LSTM(64)))
model.add(BatchNormalization())
model.add(Dropout(DROPOUT_RATE))

# model.add(Bidirectional(LSTM(16, return_sequences=True)))
# model.add(Dropout(DROPOUT_RATE))
# model.add(BatchNormalization())
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(DROPOUT_RATE))

model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss=losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 350, 100)          5000100   
_________________________________________________________________
bidirectional (Bidirectional (None, 350, 128)          84480     
_________________________________________________________________
batch_normalization (BatchNo (None, 350, 128)          512       
_________________________________________________________________
dropout (Dropout)            (None, 350, 128)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [9]:
# require this else the model.fit hits an error

y_train = np.array(y_train)

In [10]:
earlystop_cb = [EarlyStopping(
        monitor="val_loss",
        min_delta=1e-2,
        patience=3,
        verbose=1,
    )
]

# fit the model
model.fit(padded_docs, 
          y_train,         
          #validation_data = (X_test, y_test),
          epochs=10,
          batch_size=128,
          callbacks=earlystop_cb,
          validation_split=0.25,
          verbose=1)

# evaluate the model
loss, accuracy = model.evaluate(padded_docs, y_train, verbose=1)
print('Accuracy: %f' % (accuracy*100))

SyntaxError: invalid syntax (<ipython-input-10-8cc14d781c07>, line 16)